In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
procedureDF = pd.read_csv('procedure.csv')

In [2]:
procedureDF['procedure_datetime']=pd.to_datetime(procedureDF['procedure_datetime'])
procedureDF['visit_start_datetime']=pd.to_datetime(procedureDF['visit_start_datetime'])

In [3]:
persondrop=['location_id','provider_id','care_site_id','person_source_value','race_source_value',
            'gender_source_value','ethnicity_source_value','birth_datetime']
visitdrop=['visit_source_value','visit_start_date','visit_end_date','visit_end_datetime']
proceduredrop=['procedure_source_value','procedure_source_concept_id','procedure_date']
procedureDF=procedureDF.drop(columns=persondrop)
procedureDF=procedureDF.drop(columns=visitdrop)
procedureDF=procedureDF.drop(columns=proceduredrop)

In [4]:
procedureDF.loc[pd.isnull(procedureDF['death_date']) , 'death'] = 0
procedureDF.loc[pd.isnull(procedureDF['death_date'])==False , 'death'] = 1
deathdrop=['death_date','death_datetime','death_type_concept_id','cause_source_concept_id']
procedureDF=procedureDF.drop(columns=deathdrop)

In [5]:
newprocedureDF=procedureDF[procedureDF['visit_start_datetime'] == procedureDF.groupby('person_id')['visit_start_datetime'].transform('max')]

In [6]:
percent_missing = newprocedureDF.isnull().sum() * 100 / len(newprocedureDF)
missing_value_df = pd.DataFrame({'column_name': newprocedureDF.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
person_id,person_id,0.000000
visit_occurrence_id,visit_occurrence_id,0.000000
year_of_birth,year_of_birth,0.000000
ethnicity_concept_id,ethnicity_concept_id,0.000000
month_of_birth,month_of_birth,0.000000
day_of_birth,day_of_birth,0.000000
race_concept_id,race_concept_id,15.276843
gender_concept_id,gender_concept_id,0.000000
visit_start_datetime,visit_start_datetime,0.000000
visit_concept_id,visit_concept_id,0.000000


In [7]:
newprocedureDF = newprocedureDF.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [8]:
categorical= ['person_id','visit_occurrence_id','ethnicity_concept_id','race_concept_id','gender_concept_id',
             'visit_concept_id','visit_type_concept_id','procedure_concept_id','procedure_occurrence_id']
for col in categorical:
    newprocedureDF[col] = newprocedureDF[col].astype('object')
newprocedureDF['death'] = newprocedureDF['death'].astype(int)

In [9]:
newprocedureDF.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

person_id                    97916
visit_occurrence_id          94834
ethnicity_concept_id             2
race_concept_id                  6
gender_concept_id                2
visit_concept_id                 3
visit_type_concept_id            1
procedure_concept_id          6242
procedure_occurrence_id    2529375
dtype: int64

In [9]:
newprocedureDF3=newprocedureDF.drop(columns=['visit_start_datetime','visit_type_concept_id','procedure_occurrence_id','procedure_datetime'])

In [12]:
newprocedureDF4=newprocedureDF3.groupby('person_id')['procedure_concept_id'].agg({'count':len})

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [15]:
newprocedureDF4=newprocedureDF4.rename(columns={'count': 'procedure_num'})

In [17]:
newprocedureDF4.to_csv('finalprocedure.csv')